In [1]:
# Combining with structured transforms
import karps as ks
import karps.functions as f
from karps.display import show_phase

In [2]:
# Make a session
s = ks.session("demo2e")

In [3]:
employees = ks.dataframe([
    ("ACME", "John", "12/01"),
#    ("ACME", "Kate", "09/04"),
#    ("ACME", "Albert", "09/04"),
#    ("Databricks", "Ali", "09/04"),
], schema=["company_name", "employee_name", "dob"],
   name="employees")
employees

/employees@org.spark.DistributedLiteral:{company_name:string, employee_name:string, dob:string}

In [4]:
# Now use this to group by companies:
res = (employees.dob
       .groupby(employees.company_name)
       .agg({
           "agg1": f.count,
           "agg2": f.count
       }))
o = f.collect(res)
o

/collect_list8!org.spark.StructuredReduce:[{company_name:string, agg1:int, agg2:int}]

In [5]:
s.run(o)

_Rendezvous: <_Rendezvous of RPC that terminated with (StatusCode.UNKNOWN, )>

In [ ]:
# The number of people who share a birthday date with someone else.
# Takes a column of data containing birthdates.
def paradoxal_count(c):
    with ks.scope("p_count"): # Make it pretty:
        g = c.groupby(c).agg({'num_employees': f.count}, name="agg_count")
        s = f.sum(g.num_employees[g.num_employees>=2], name="paradoxical_employees")
        return s


In [ ]:
df = ks.dataframe(["1/1", "3/1", "1/1"])
s.run(paradoxal_count(df))

In [ ]:
# Now use this to group by companies:
res = (employees.dob
       .groupby(employees.company_name)
       .agg({
           "paradoxical_employees": paradoxal_count
       }))
o = f.collect(res)
o

In [ ]:
comp = s.compute(o)

In [ ]:
show_phase(comp, "initial")

In [ ]:
show_phase(comp, "final")

In [ ]:
comp.values()